<a href="https://colab.research.google.com/github/PUC-RecSys-Class/RecSysPUC-2022/blob/master/practicos/Content_Based_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Práctico Content-based (Texto)

**Profesor:** Denis Parra

**Ayudantes:** Álvaro Labarca, Jorge Facuse, Florencia Ferrer, Nicolás Sumonte, Pablo Messina 


En este proyecto trabajaremos con un modelo de recomendacion de libros de la página [Goodreads](http://www.goodreads.com). El modelo de recomendación de libros es un recomendador basado en contenido, donde se utilizan modelos de lenguage BERT y BERT-large para el cálculo de embeddings de los libros y luego similaridades de ítems. Luego, dependiendo de los libros con los que el usuario ha interactuado, se recomiendan los ítems más similares.

In [1]:
import numpy as np
import json
import requests
import heapq
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from io import BytesIO
import pickle
import pandas as pd
import time 

Descargamos datos que vienen previamente calculados: 
- transacciones/interacciones de cada usuario 
- transaciones para evaluar el modelo 
- embeddings de descripciones calculados con BERT  
- embeddings de descripciones calculados con BERT-large
- datos de libros con información de titulo, descripcion, año de publicacion, entre otros. 

In [2]:
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

--2022-09-07 19:33:17--  https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/57tel5zqopkssrh/books.csv [following]
--2022-09-07 19:33:17--  https://www.dropbox.com/s/raw/57tel5zqopkssrh/books.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8e8da3f507b0027c6260cd9756.dl.dropboxusercontent.com/cd/0/inline/BsjWuOfNkW19QWbqfA5ZM4rVNO-iwvqfpLT9ZxQExSu8vH9FdI6tCZCImrWlrGT7VwkS-RSjp6EMqH2vC8aw0CzU_-NHS83DNLi9wh01taovWbV7ovdYz9IMesqHthC4YBTu4PeDsmnmoY8i22MgjRvNWLXQ8cufyErT4oCHRfzcIQ/file# [following]
--2022-09-07 19:33:17--  https://uc8e8da3f507b0027c6260cd9756.dl.dropboxusercontent.com/cd/0/inline/BsjWuOfNkW19QWbqfA5ZM4rVNO-iwvqfpLT9ZxQExSu8vH9FdI6tCZCImrWlrGT7VwkS-RSjp6EMqH2vC8aw

# Cargar datos adicionales

In [3]:
df_books = pd.read_csv('books.csv', sep=',')
df_books.head()


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [4]:
# diccionario con id del usuario y id de libros con los que ha interactuado en el pasado 
with open('goodreads_past_interactions.json') as f:
    user_interactions = json.load(f)

# diccionario con id del usuario y id de libros para testear el modelo     
with open('goodreads_test_interactions.json') as f:
    user_interactions_test = json.load(f)


In [5]:
# dict index 2 book id and vice-versa for recommendation 
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

# Cargar características pre-entrenadas: BERT y BERT-large

En esta sección se trabajará con modelos pre-entrenados de modelos de lenguage BERT y BERT-large que convierten texto a embeddings. 

Bidirectional Encoder Representations from Transformers (BERT) es una técnica de NLP (Natural Language Processing) desarrollada por Google y publicada en 2018 por Jacob Devlin. 

Actualmente Google utiliza BERT para entender las consultas de los usuarios en su buscador. 

Tiene dos versiones: 
- **BERT:** 12 capas, 12 cabezales de atencion y 110 millones de parámetros. Genera vectores de 768 dimensiones 
- **BERT-large:** 24 capas, 16 cabezales de atencion y 340 millones de parámetros.  

![BERT y BERT-large](http://jalammar.github.io/images/bert-base-bert-large.png)

![BERT y BERT-large arquitectura](http://jalammar.github.io/images/bert-base-bert-large-encoders.png)

En este caso los textos que utilizaremos son los títulos de los libros con su descripción y compararemos los resultados de recomendación con BERT y BERT-large. Para efectos de este trabajo los vectores de características ya fueron entrenados y guardados en archivos numpy. A continuación son cargados en memoria.

Para mayores detalles sobre el modelo de lenguaje BERT se recomienda revisar el siguiente artículo:
- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)


In [6]:
bert_featmat = np.load('goodreads_bert_embeddings.npy', allow_pickle=True)
bert_large_featmat = np.load('goodreads_bert_large_embeddings.npy', allow_pickle=True)

In [7]:
bert_featmat.shape

(4287, 768)

In [8]:
bert_large_featmat.shape

(4287, 1024)

### **Pregunta 1** 

Considerando que haremos un recomendador basado en contenidos ¿Por qué el uso de modelos de lenguage es una buena elección para este tipo de problema?

**Respuesta 1:**

# Probamos con BERT y BERT-large reduciendo dimensionalidad con PCA-20

Una vez calculado (o cargado) los vectores característicos de cada libro a partir de su titulo y descripción, reducimos dimensionalidad. Probaremos con BERT y BERT-large para comparar los resultados de ambos en recomendación basada en contenido. 


In [10]:
# Project into a 20 PCA feature space
pca20_bert_featmat = PCA(n_components=20).fit_transform(bert_featmat)
pca20_bert_large_featmat = PCA(n_components=20).fit_transform(bert_large_featmat)

In [11]:
pca20_bert_featmat.shape

(4287, 20)

In [12]:
pca20_bert_large_featmat.shape

(4287, 20)

### **Pregunta 2**

Comente por qué se utiliza PCA para reducir la dimensión de cada vector característico. ¿Qué sucede con la pérdida de información en la reducción de dimensionalidad?

### Respuesta 2: 

# Similar document retrieval 

En esta sección utilizaremos los vectores cargados para hacer un sistema de recuperación o búsqueda de información, para diferentes métricas de distancia.

Buscamos libros similares de acuerdo a la representación vectorial (BERT) de su título y descripción. 


In [13]:
# format results 
pd.options.display.max_colwidth = 50
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# Find similar images by image id
def find_similar_books(embedding, query_id=None, metric='euclidean', topk=10):
    
    n = embedding.shape[0]
    
    if query_id is None:
        query_i = np.random.randint(n)
        query_id = idx2bookid[query_i]
    
    else:
        query_i = bookid2idx[query_id]
        
    
    distances = pairwise_distances(embedding[query_i].reshape(1,-1), embedding, metric=metric)
    heap = []
    for i in range(n):            
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[0][i], i))
        else:
            heapq.heappushpop(heap, (-distances[0][i], i))

    heap.sort(reverse=True)
    rec_ids = [idx2bookid[i] for _,i in heap]
    
    return rec_ids

## Usando BERT 

In [15]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(bert_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

[3, 2908, 3115, 2303, 7334, 7235, 5721, 3510, 9552, 9696]

In [16]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
2,3,Twilight,"About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part of him—and I didn't know how dominant that part might be—that thirsted for my blood.And third, I was unconditionally and irrevocably in love with him.In the first book of the Twilight Saga, internationally bestselling author Stephenie Meyer introduces Bella Swan and Edward Cullen, a pair of star-crossed lovers whose forbidden relationship ripens against the backdrop of small-town suspicion and a mysterious coven of vampires. This is a love story with bite.",Stephenie Meyer
1423,2303,Bloody Bones,"In Laurell K. Hamilton's ""New York Times"" bestselling novels, Anita Blake, vampire hunter and animator, takes a bite out of crime-of the supernatural kind. But even someone who deals with death on a daily basis can be unnerved by its power... When Branson, Missouri, is hit with a death wave-four unsolved murders-it doesn't take an expert to realize that all is not well. But luckily for the locals, Anita is an expert-in just the kinds of preternatural goings-on that have everyone spooked. And she's got an ""in"" with just the kind of creature who can make sense of the slayings: a sexy master vampire known as Jean Claude.",Laurell K. Hamilton
1708,2908,"Severed Heads, Broken Hearts","Robyn Schneider's The Beginning of Everything is a witty and heart-wrenching teen novel that will appeal to fans of books by John Green and Ned Vizzini, novels such as The Perks of Being a Wallflower, and classics like The Great Gatsby and The Catcher in the Rye.Varsity tennis captain Ezra Faulkner was supposed to be homecoming king, but that was before—before his girlfriend cheated on him, before a car accident shattered his leg, and before he fell in love with unpredictable new girl Cassidy Thorpe.As Kirkus Reviews said in a starred review, ""Schneider takes familiar stereotypes and infuses them with plenty of depth. Here are teens who could easily trade barbs and double entendres with the characters that fill John Green's novels.""Funny, smart, and including everything from flash mobs to blanket forts to a poodle who just might be the reincarnation of Jay Gatsby, The Beginning of Everything is a refreshing contemporary twist on the classic coming-of-age novel—a heart-wrenching story about how difficult it is to play the part that people expect, and how new beginnings can stem from abrupt and tragic endings.",Robyn Schneider
1808,3115,A Hunger Like No Other,"In New York Times and USA TODAY bestselling author Kresley Cole’s sizzling series, a fierce werewolf and a bewitching vampire become unlikely soul mates whose passion will test the boundaries of life and death.After enduring years of torture from the vampire horde, Lachlain MacRieve, leader of the Lykae Clan, is enraged to find the predestined mate he’s waited millennia for is a vampire. Or partly one. Emmaline Troy is a small, ethereal half Valkyrie/half vampire, who somehow begins to soothe the fury burning within him.Sheltered Emmaline finally sets out to uncover the truth about her deceased parents—until a powerful Lykae claims her as his mate and forces her back to his ancestral Scottish castle. There, her fear of the Lykae—and their notorious dark desires—ebbs as he begins a slow, wicked seduction to sate her own dark cravings.Yet when an ancient evil from her past resurfaces, will their desire deepen into a love that can bring a proud warrior to his knees and turn a gentle beauty into the fighter she was born to be?",Kresley Cole
1999,3510,"Cerulean Sins (Anita Blake, Vampire Hunter, #11)","Cerulean Sins, the eleventh entry in the hugely-popular Anita Blake series, finds everyone’s favorite vampire hunter keeping house and kicking butt.Anita Blake is trying to get her life back to “normal” after a break-up with her werewolf lover. She has settled into a pattern of domesticity, which means that the new man in her life, the leopard shapeshifte

## Usando BERT reducidos con PCA 

In [17]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(pca20_bert_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

[3, 1354, 2908, 1886, 476, 2041, 7235, 9552, 4754, 6011]

In [18]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
2,3,Twilight,"About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part of him—and I didn't know how dominant that part might be—that thirsted for my blood.And third, I was unconditionally and irrevocably in love with him.In the first book of the Twilight Saga, internationally bestselling author Stephenie Meyer introduces Bella Swan and Edward Cullen, a pair of star-crossed lovers whose forbidden relationship ripens against the backdrop of small-town suspicion and a mysterious coven of vampires. This is a love story with bite.",Stephenie Meyer
354,476,The World According to Garp,"This is the life and times of T. S. Garp, the bastard son of Jenny Fields--a feminist leader ahead of her times. This is the life and death of a famous mother and her almost-famous son; theirs is a world of sexual extremes--even of sexual assassinations. It is a novel rich with ""lunacy and sorrow""; yet the dark, violent events of the story do not undermine a comedy both ribald and robust. In more than thirty languages, in more than forty countries--with more than ten million copies in print--this novel provides almost cheerful, even hilarious evidence of its famous last line: ""In the world according to Garp, we are all terminal cases.""The World According to Garp is a comic and compassionate coming-of-age novel that established John Irving as one of the most imaginative writers of his generation. A worldwide bestseller since its publication in 1978, Irving's classic is filled with stories inside stories about the life and times of T. S. Garp, novelist and bastard son of Jenny Fields--a feminist leader ahead of her time. Beyond that, The World According to Garp virtually defies synopsis.(From author's website: http://www.john-irving.com)",John Irving
919,1354,The Strange Case of Dr. Jekyll and Mr. Hyde and Other Tales of Terror,"\r\r\r\r\r\r\n'He put the glass to his lips and drank at one gulp... his face became suddenly black and the features seemed to melt and alter'\r\r\r\r\r\r\nPublished as a 'shilling shocker', Robert Louis Stevenson's dark psychological fantasy gave birth to the idea of the split personality. The story of respectable Dr Jekyll's strange association with 'damnable young man' Edward Hyde; the hunt through fog-bound London for a killer; and the final revelation of Hyde's true identity is a chilling exploration of humanity's basest capacity for evil. The other stories in this volume also testify to Stevenson's inventiveness within the Gothic tradition: ""Olalla', a tale of vampirism and tainted family blood, and 'The Body Snatcher', a gruesome fictionalisation of the exploits of the notorious Burke and Hare.This edition contains a critical introduction by Robert Mighall, which discusses class, criminality and the significance of the story's London setting. It also includes an essay on the scientific contexts of the novel and the development of the idea of the Jekyll-and-Hyde personality.","Robert Louis Stevenson, Robert Mighall"
1201,1886,Preacher Vol. 1: Gone to Texas,"One of the most celebrated comics titles of the late 1990s, PREACHER is a modern American epic of life, death, love and redemption also packed with sex, booze, blood and bullets - not to mention angels, demons, God, vampires and deviants of all stripes.At first glance, the Reverend Jesse Custer doesn't look like anyone special-just another small-town minister slowly losing his flock and his faith. But he's about to come face-to-face with proof that God does indeed exist. Merging with a bizarre spiritual force called Genesis, Jesse now possesses the power of ""the Word,"" an ability to make people do whatever he utters. He begins a violent and riotous journey across the country in search of answers from the elusive deity.","Garth Ennis, Steve Dillon, Joe R. Lansdale"
1289,2041,Lisey's Story,"Lisey Landon lost her husband Scott two years ago, after a twenty-five-year marriage of profoun

## Usando BERT-large

In [19]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(bert_large_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

[3, 7818, 712, 5118, 1818, 7221, 2112, 7101, 8266, 7552]

In [20]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
2,3,Twilight,"About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part of him—and I didn't know how dominant that part might be—that thirsted for my blood.And third, I was unconditionally and irrevocably in love with him.In the first book of the Twilight Saga, internationally bestselling author Stephenie Meyer introduces Bella Swan and Edward Cullen, a pair of star-crossed lovers whose forbidden relationship ripens against the backdrop of small-town suspicion and a mysterious coven of vampires. This is a love story with bite.",Stephenie Meyer
527,712,"Drums of Autumn (Outlander, #4)","In this breathtaking novel—rich in history and adventure—The New York Times bestselling author Diana Gabaldon continues the story of Claire Randall and Jamie Fraser that began with the now-classic novel Outlander and continued in Dragonfly in Amber and Voyager. Once again spanning continents and centuries, Diana Gabaldon has created a work of sheer passion and brilliance.... It began at an ancient Scottish stone circle. There, a doorway, open to a select few, leads into the past—or the grave. Dr. Claire Randall survived the extraordinary passage, not once but twice.Her first trip swept her into the arms of Jamie Fraser, an eighteenth-century Scot whose love for her became a legend—a tale of tragic passion that ended with her return to the present to bear his child. Her second journey, two decades later, brought them together again in the American colonies. But Claire had left someone behind in the twentieth century—their daughter, Brianna....Now Brianna has made a disturbing discovery that sends her to the circle of stones and a terrifying leap into the unknown. In search of her mother and the father she has never met, she is risking her own future to try to change history ... and to save their lives. But as Brianna plunges into an uncharted wilderness, a heartbreaking encounter may strand her forever in the past ... or root her in the place she should be, where her heart and soul belong....",Diana Gabaldon
1160,1818,One Foot in the Grave,"You can run from the grave, but you can’t hide…Half-vampire Cat Crawfield is now Special Agent Cat Crawfield, working for the government to rid the world of the rogue undead. She’s still using everything Bones, her sexy and dangerous ex, taught her, but when Cat is targeted for assassination, the only man who can help her is the vampire she left behind.Being around Bones awakens all her emotions, from the adrenaline rush of slaying vamps side by side to the reckless passion that consumed them. But a price on her head—wanted: dead or half-alive—means her survival depends on teaming up with Bones. And no matter how hard Cat tries to keep things professional between them, she’ll find that desire lasts forever…and Bones won’t let her get away again.",Jeaniene Frost
1321,2112,P.S. I Still Love You,"Lara Jean didn’t expect to really fall for Peter.She and Peter were just pretending. Except suddenly they weren’t. Now Lara Jean is more confused than ever.When another boy from her past returns to her life, Lara Jean’s feelings for him return too. Can a girl be in love with two boys at once?In this charming and heartfelt sequel to the New York Times bestseller To All the Boys I've Loved Before, we see first love through the eyes of the unforgettable Lara Jean. Love is never easy, but maybe that’s part of what makes it so amazing.",Jenny Han
2672,5118,Stars Above,"The enchantment continues....The universe of the Lunar Chronicles holds stories—and secrets—that are wondrous, vicious, and romantic. How did Cinder first arrive in New Beijing? How did the brooding soldier Wolf transform from young man to killer? When did Princess Winter and the palace guard Jacin realize their destinies?With nine stories—five of which have never before been published—and an exclusive never-before-seen excerpt from Marissa Meyer’s upcoming novel, Heartless, about

## Usando BERT-large reducidos con PCA 

In [21]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(pca20_bert_large_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

[3, 1818, 6977, 3549, 3984, 7818, 7221, 2476, 2483, 3408]

In [22]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
2,3,Twilight,"About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part of him—and I didn't know how dominant that part might be—that thirsted for my blood.And third, I was unconditionally and irrevocably in love with him.In the first book of the Twilight Saga, internationally bestselling author Stephenie Meyer introduces Bella Swan and Edward Cullen, a pair of star-crossed lovers whose forbidden relationship ripens against the backdrop of small-town suspicion and a mysterious coven of vampires. This is a love story with bite.",Stephenie Meyer
1160,1818,One Foot in the Grave,"You can run from the grave, but you can’t hide…Half-vampire Cat Crawfield is now Special Agent Cat Crawfield, working for the government to rid the world of the rogue undead. She’s still using everything Bones, her sexy and dangerous ex, taught her, but when Cat is targeted for assassination, the only man who can help her is the vampire she left behind.Being around Bones awakens all her emotions, from the adrenaline rush of slaying vamps side by side to the reckless passion that consumed them. But a price on her head—wanted: dead or half-alive—means her survival depends on teaming up with Bones. And no matter how hard Cat tries to keep things professional between them, she’ll find that desire lasts forever…and Bones won’t let her get away again.",Jeaniene Frost
1492,2476,The Winter Sea,"In the spring of 1708, an invading Jacobite fleet of French and Scottish soldiers nearly succeeded in landing the exiled James Stewart in Scotland to reclaim his crown.Now, Carrie McClelland hopes to turn that story into her next bestselling novel. Settling herself in the shadow of Slains Castle, she creates a heroine named for one of her own ancestors and starts to write.But when she discovers her novel is more fact than fiction, Carrie wonders if she might be dealing with ancestral memory, making her the only living person who knows the truth—the ultimate betrayal—that happened all those years ago, and that knowledge comes very close to destroying her.…","Susanna Kearsley, Rosalyn Landor"
1496,2483,Rush,"Gabe, Jace, and Ash: three of the wealthiest, most powerful men in the country. They’re accustomed to getting anything they want. Anything at all. For Gabe, it’s making one particular fantasy come true with a woman who was forbidden fruit. Now she’s ripe for the picking...When Gabe Hamilton saw Mia Crestwell walk into the ballroom for his hotel’s grand opening, he knew he was going to hell for what he had planned. After all, Mia is his best friend’s little sister. Except she’s not so little anymore. And Gabe has waited a long time to act on his desires.Gabe has starred in Mia’s fantasies more than once, ever since she was a teenager with a huge crush on her brother’s best friend. So what if Gabe’s fourteen years older? Mia knows he’s way out of her league, but her attraction has only grown stronger with time. She’s an adult now, and there’s no reason not to act on her most secret desires.As Gabe pulls her into his provocative world, she realizes there’s a lot she doesn’t know about him or how exacting his demands can be. Their relationship is intense and obsessive, but as they cross the line from secret sexual odyssey to something deeper, their affair runs the risk of being exposed—and vulnerable to a betrayal far more intimate than either expected.",Maya Banks
1944,3408,Hunting Ground,"Anna Latham didn’t know how complicated life could be until she became a werewolf. And until she was mated to Charles Cornick, the son — and enforcer — of Bran, the leader of the North American werewolves, she didn’t know how dangerous it could be either...Anna and Charles have just been enlisted to attend a summit to present Bran’s controversial proposition: that the wolves should finally reveal themselves to humans. But the most feared Alpha in Europe is dead set against the plan — and it seems like someon

### Pregunta 3: 
Comente los resultados obtenidos, en cuanto a modelo de lenguaje, reduccion de dimensionalidad y métrica de distancia utilizada. 


### Respuesta 3: 

# Recomendaciones 

In [23]:
# format results 
pd.options.display.max_colwidth = 50
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


## Función para obtener recomendacion para cada usuario 

In [24]:
def recommend(embedding, user_id=None, topk=10, metric='cosine'):
    
    #print("user_id = ", user_id)
    
    user_id = str(user_id)
    
    #Calculate distance metrics
    trx = user_interactions[user_id]
    n = embedding.shape[0]
    distances = 1e9
    
    # recorremos transacciones pasadas del usuario 
    for t in trx:
        query_i = bookid2idx[t]
        
        # recomendamos items más cercanos a items con los que interactuó el usuario
        distances = np.minimum(distances, pairwise_distances(
                embedding[query_i].reshape(1,-1), embedding, metric=metric).reshape(-1))

    #Rank items de menor a mayor distancia (nos quedamos con los topk)
    trx_set = set(trx)
    heap = []
    for i in range(n):
        if idx2bookid[i] in trx_set:
            continue
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[i], i))
        else:
            heapq.heappushpop(heap, (-distances[i], i))
    heap.sort(reverse=True)
    
    # utilizamos un heap para extraer los items ordenados de menor a mayor distancia 
    recommended_ids = [idx2bookid[i] for _,i in heap]
    
    # retornar los que el usuario no haya consumido 
    filtered_recommended_ids = []
    
    return recommended_ids

## generar recomendaciones para un usuario en particular

In [25]:
# recomendación para el usuario id = 49299 , utilizando bert con reduccion de dimensionalidad a 20 
user_id = '50101'
rec = recommend(pca20_bert_featmat, user_id=user_id, topk=15)
rec 

[4509,
 5002,
 4376,
 964,
 2292,
 7937,
 5126,
 9473,
 5244,
 390,
 6219,
 7602,
 7913,
 6865,
 2796]

## transacciones pasadas del usuario 

In [26]:
past_interactions = user_interactions[str(user_id)]

df_books[df_books['book_id'].isin(past_interactions)][['book_id', 'original_title', 'book_desc', 'authors']]


,book_id,original_title,book_desc,authors
2,3,Twilight,"About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part of him—and I didn't know how dominant that part might be—that thirsted for my blood.And third, I was unconditionally and irrevocably in love with him.In the first book of the Twilight Saga, internationally bestselling author Stephenie Meyer introduces Bella Swan and Edward Cullen, a pair of star-crossed lovers whose forbidden relationship ripens against the backdrop of small-town suspicion and a mysterious coven of vampires. This is a love story with bite.",Stephenie Meyer
5,6,The Fault in Our Stars,"Despite the tumor-shrinking medical miracle that has bought her a few years, Hazel has never been anything but terminal, her final chapter inscribed upon diagnosis. But when a gorgeous plot twist named Augustus Waters suddenly appears at Cancer Kid Support Group, Hazel's story is about to be completely rewritten.Insightful, bold, irreverent, and raw, The Fault in Our Stars is award-winning author John Green's most ambitious and heartbreaking work yet, brilliantly exploring the funny, thrilling, and tragic business of being alive and in love.",John Green
7,8,The Catcher in the Rye,"The hero-narrator of The Catcher in the Rye is an ancient child of sixteen, a native New Yorker named Holden Caulfield. Through circumstances that tend to preclude adult, secondhand description, he leaves his prep school in Pennsylvania and goes underground in New York City for three days. The boy himself is at once too simple and too complex for us to make any final comment about him or his story. Perhaps the safest thing we can say about Holden is that he was born in the world not just strongly attracted to beauty but, almost, hopelessly impaled on it. There are many voices in this novel: children's voices, adult voices, underground voices-but Holden's voice is the most eloquent of all. Transcending his own vernacular, yet remaining marvelously faithful to it, he issues a perfectly articulated cry of mixed pain and pleasure. However, like most lovers and clowns and poets of the higher orders, he keeps most of the pain to, and for, himself. The pleasure he gives away, or sets aside, with all his heart. It is there for the reader who can handle it to keep. J.D. Salinger's classic novel of teenage angst and rebellion was first published in 1951. The novel was included on Time's 2005 list of the 100 best English-language novels written since 1923. It was named by Modern Library and its readers as one of the 100 best English-language novels of the 20th century. It has been frequently challenged in the court for its liberal use of profanity and portrayal of sexuality and in the 1950's and 60's it was the novel that every teenage boy wants to read.",J.D. Salinger
8,10,Pride and Prejudice,"«È cosa ormai risaputa che a uno scapolo in possesso di un'ingente fortuna manchi soltanto una moglie. Questa verità è cosí radicata nella mente delle famiglie del luoho che, nel momento in cui un simile personaggio viene a far parte del vicinato, prima ancora di conoscere anche lontanamente i suoi desiderî in proposito, viene immediatamente considerato come proprietà legittima di una o l'altra delle loro figlie.»Orgoglio e pregiudizio è uno dei primi romanzi di Jane Austen. La scrittrice lo iniziò a ventun anni; il libro, rifiutato da un editore londinese, rimase in un cassetto fino alla sua pubblicazione anonima nel 1813, e da allora è considerato tra i piú importanti romanzi della letteratura inglese. È la storia delle cinque sorelle Bennet e dei loro corteggiatori, con al centro il romantico contrasto tra l'adorabile e capricciosa Elizabeth e l'altezzoso Darcy; lo spirito di osservazione implacabile e quasi cinico, lo studio arguto dei caratteri, la satira delle vanità e delle debolezze della vita domestica, fanno di questo romanzo una delle piú efficaci e indimenticabili commedie di costume del periodo Regency inglese.

## información de recomendaciones 

In [27]:
df_books[df_books['book_id'].isin(rec)][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
293,390,The Strange Case of Dr Jekyll and Mr Hyde,"Un abogado, Gabriel John Utterson, investiga la extraña relación entre su viejo amigo, el doctor. Henry Jekyll, y el misántropo Edward Hyde. En realidad, Jekyll es un científico que crea una poción o bebida que tiene la capacidad de separar la parte más humana del lado más maléfico de una persona. Cuando Jekyll bebe esta mezcla se convierte en Edward Hyde, un criminal capaz de cualquier atrocidad. Desesperado por la necesidad cada vez más frecuente e incontrolable de convertirse en ese ser malvado, el doctor Jekyll decide suicidarse.","Robert Louis Stevenson, Vladimir Nabokov, Mervyn Peake, Dan Chaon"
683,964,The Hobbit and The Lord of the Rings,لجزء الثالث من ملحمة جيه أر أر تولكين الرائعة سيد الخواتم دخل أفراد رفقة الخاتم فى مغامرات منفصلة أثناء إنجاز المهمة وسيرها فى طريق التنفيذ - اما أراجون الذى كشف أنه الوريث الخلفى لملوك الغرب القدامى فقد أنضم إلى خيالة روهان ضد قوى أيزنجارد وأشترك فى نصر هورنبرج الذى تحقق بصعوبة وبأس - أما ميرى وبيبين كانا قد تعرضا للإختطاف من جانب الأوركيين فقد هربا إلى غابة فانجون وهناك قابلا الأنتيين وعاد جاندالف فيما يشبة المعجزة وهزم الساحر الشرير سارومان وفى نفس الوقت فإن سام وفرودو راحا يتقدمان بإتجاه موردو لتدمير الخاتم وبرفقتهما سميجول - جولام وهو لا يزال مهووسا ب شيئة الثمين وبعد معركة مع أنثى العنكبوت العملاق شيلوب ترك سام سيده طنا منه أنه مات ولكن فرودو كان لا يزال على قيد الحياة فى أيدى الأوركيين وكانت جيوش سيد الظلام تتجمع بإستمرار,J.R.R. Tolkien
1417,2292,ساق البامبو [Saq al-Bambu],لماذا كان جلوسي تحت الشجرة يزعج أمي؟ أتراها كانت تخشى أن تنبت لي جذور تضرب في عمق الأرض ما يجعل عودتي إلى بلاد أبي أمراً مستحيلاً؟..ربما، ولكن، حتى الجذور لا تعني شيئاً أحياناً. لو كنت مثل نبتة البامبو.. لا انتماء لها.. نقتطع جزءاً من ساقها.. نغرسه، بلا جذور، في أي أرض.. لا يلبث الساق طويلاً حتى تنبت له جذور جديدة.. تنمو من جديد.. في أرض جديدة.. بلا ماض.. بلا ذاكرة.. لا يلتفت إلى اختلاف الناس حول تسميته.. كاوايان في الفلبين.. خيزران في الكويت.. أو بامبو في أماكن أخرى.,"سعود السنعوسي, Saud Alsanousi"
1643,2796,The Lost Wife,"Praga, 1938. Lenka, una joven estudiante de arte, se enamora perdidamente de Josef. Se casan, pero a pesar de esperar un prometedor futuro juntos, sus sueños se hacen añicos ante la inminente invasión nazi y deciden huir a Estados Unidos. Sin embargo, en el último momento, Lenka decide quedarse, y los amantes se separan con la promesa de reunirse tan pronto como puedan.En Estados Unidos, Josef rehace su vida, aunque nunca olvidará a su primer amor, al que cree haber perdido para siempre. Mientras, en medio del horror de Tezerín y de Auschwitz, Lenka sobrevivirá gracias a la memoria de un marido al que cree que no volverá a ver y a su talento artístico, la única vía que da color al horror y forma a lo que no se puede describir con palabras.Separados por la guerra, pero unidos por el recuerdo, ahora, décadas más tarde, un inesperado encuentro en Nueva York les ofrece una segunda oportunidad..."".",Alyson Richman
2363,4376,شيكاجو,يقول الأستاذ جلال أمين عن هذه الرواية المتميزة: ها هى رواية علاء الأسوانى الجديدة «شيكاجو» تستحق بدورها نجاحًا مماثلاً وبنفس القدر من الجدارة كالذى استحقته عمارة يعقوبيان. فرحت عند انتهائى من قراءتها لأكثر من سبب، فقد أكدت لى هذه القراءة أن لدينا بالفعل أديبًا كبيرًا وموهوبًا، وظهر أن عمارة يعقوبيان ليست ظاهرة منفردة لا تتكرر، بل إن من الممكن أن تتكرر المرة بعد المرة. فى الرواية الجديدة (شيكاجو) كل مزايا الرواية السابقة: التشويق الذى يبدأ من أول صفحة ويستمر إلى آخر صفحة، أسلوب الكتابة السلس والسريع الذى يصيب الهدف باستمرار بلا تثاقل أو تسكع، الرسم الواضح والمتسق للشخصيات، اللغة العربية الراقية دون تكلف أو تعمد الإغراب، وقبل كل شيء وفوق كل شيء، نُبل المعني، إذ لا جدوى فى رأيى من رواية مهما كانت درجة تشويقها وإتقانها إذا لم تكن نبيلة المقصد، وإذا كان المقصد تافهًا أو حقيرًا قضى على ما قد يكون للمهارة والشطارة من أثر فى نفس القارئhttp://www.titaniumstores.com/dp/9770...,"Alaa Al Aswany, علاء الأسواني"
2418,4509,2666,"A cuatro profesores de literatura, Pelletier, Morini, Espinoza 

# evaluación de las recomendaciones con interacciones de testing 

In [28]:
# Métricas de evaluación 
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])
  
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg


## Evaluación de recomendación con BERT

In [29]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

embeddings = pca20_bert_featmat
topk = 10 

for i, u in enumerate(user_interactions_test.keys()):
    
    print(i, end= '\r')
    
    rec = recommend(bert_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [30]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))


MAP  0.03669047619047619
ndcg@10 0.13
tiempo de ejecucion 258.76 segs


## Evaluación de recomendación con BERT reducidos con PCA-20

In [31]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

embeddings = pca20_bert_featmat
topk = 10 

for i, u in enumerate(user_interactions_test.keys()):
    
    print(i, end= '\r')
    
    rec = recommend(pca20_bert_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [32]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.00453968253968254
ndcg@10 0.03
tiempo de ejecucion 20.62 segs


## Evaluación de recomendación con BERT-large

In [33]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10 

for i, u in enumerate(user_interactions_test.keys()):
    
    print(i, end= '\r')
    
    rec = recommend(bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start

In [34]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.060920634920634924
ndcg@10 0.24
tiempo de ejecucion 343.16 segs


## Evaluación de recomendación con BERT-large reducidos con PCA-20

In [ ]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10 

for i, u in enumerate(user_interactions_test.keys()):
    
    print(i, end= '\r')
    
    rec = recommend(pca20_bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [37]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.026123015873015874
ndcg@10 0.07
tiempo de ejecucion 17.42 segs


### Pregunta 4: 
- Comente los resultados en términos de tiempo de ejecución y métricas de ranking para los 4 modelos. 

### Respuesta 4: